<h1><Center>Google Trends Analytics using Shiny</Center></h1>

# Lesson Goals

<div style="text-align: Justify"> In this lesson, we will use Google Trends API and write R program that fetches the trends of many search terms and do spatial comparison where the searches are performed. </div>

______
# Prerequests 
  1. Install gtrendsR
  
  NB:make sure Jupyter Notebook 

UI.R

In [ ]:
library(shiny)
library(shinydashboard)

dashboardPage(
  dashboardHeader(title="By Ashenafi"),
  
  dashboardSidebar(
    br(),
    
    
    h6(" Search Term(s)",style="text-align:center;color:#FFA319;font-size:150%"),
    
    helpText("Give one or more terms that you want R to retrieve data from the Google Trends API.
             Use comma to separate terms", style="text-align:center"),
    
    textInput('terms',''),
    
    
    selectInput("geography", 
                label = tags$h4(strong(em("Geography")),style="text-align:center;color:#FFA319;font-size:150%"),
                choices = c("Worldwide"),
                selected = "United States"),           
    selectInput("period", 
                label = tags$h4(strong(em("Time Period")),style="text-align:center;color:#FFA319;font-size:150%"),
                choices = c("2016-present",
                            "Past30Days",
                            "Past90Days",
                            "Past12Months",
                            "2016",
                            "2017",
                            "2018",
                            "2019",
                            "2020"
                ),
                selected = "2016-present"),
    
    checkboxInput("corr", 
                  label = strong("Correlation",style="text-align:center;color:#FFA319;font-size:150%")),
    br(),
    
    tags$h1(submitButton("Update!"),style="text-align:center"),
    helpText("To get results, click the 'Update!' button",style="text-align:center"),
    
    br(),
    br(),
    br(),
    br(),
    br(),
    br()
    
    
    
  ),
  
  
  #####
  ##  Main Panel
  #### help ====        
  dashboardBody(    
    fluidRow(
      br(),
      h5(em(strong("Google Trends Analytics", style="color:darkblue;font-size:210%")),align = "center"),
      
      plotOutput("myplot"),
      br(),
      plotOutput("myplot3"),
      plotOutput("myplot2")
      
      
    )
  ))

Server.R

In [ ]:
# Load libraries ====
if(!require(shiny)){
  install.packages('shiny')
}
if(!require(gtrendsR)){
  install.packages('gtrendsR')
}
if(!require(reshape2)){
  install.packages('reshape2')
}
if(!require(ggplot2)){
  install.packages('ggplot2')
}

library(shiny)
library(gtrendsR)
library(reshape2)
library(ggplot2)

data(countries)

# Start shiny application

shinyServer(function(input, output) {
  
  
  
  out <- reactive({
    if(length(input$terms)>0){
      
      unlist(strsplit(input$terms,","))
    }
  })
  
  start_date<-reactive({
    
    if(input$period=="2016-present"){as.Date("2016-01-01")}
    
    else if (input$period=="Past90Days"){as.Date(Sys.time())-90}
    
    else if (input$period=="Past12Months"){
      m=as.POSIXlt(as.Date(Sys.time()))
      m$year=m$year-1
      m}
    
    else if (input$period=="2016"){as.Date("2016-01-01")}
    else if (input$period=="2017"){as.Date("2017-01-01")}
    else if (input$period=="2018"){as.Date("2018-01-01")}
    else if (input$period=="2019"){as.Date("2019-01-01")}
    else if (input$period=="2020"){as.Date("2020-01-01")}
    
    
    
  })
  
  
  end_date<-reactive({
    
    if(input$period %in% c("2016-present",
                           "Past90Days","Past12Months"))
                 {
      as.Date(Sys.time())}
      
    else if (input$period=="2016"){as.Date("2016-12-31")}
    else if (input$period=="2017"){as.Date("2017-12-31")}
    else if (input$period=="2018"){as.Date("2018-12-31")}
    else if (input$period=="2019"){as.Date("2019-12-31")}
    else if (input$period=="2020"){as.Date(Sys.time())} 
      
    })
  
  geo<-reactive({
    if(input$geography=="Worldwide"){""}
    
    else{
      
      countries$CODE[countries$COUNTRY==input$geography]
    }
    
  })
  
  data<-reactive({
    if(length(out()>0))
    {
      
      #out2<-gtrends(query=out(),start_date=start_date(),end_date=end_date(),geo=geo())
      out2<-gtrends(keyword = out(),
                   time = paste0(start_date()," ",end_date()),geo=geo())
      
    }
    
  })
  
  
  
  
  output$myplot <- renderPlot({
    if(length(out()>0)){
      z=data()
      trend=z$interest_over_time
      
      #if("end"%in%names(date)==T)
      #{
       # trend=select(trend,-end)}
      
      #trend <- melt(trend, id='start')
      
      ggplot(trend, aes(date,hits, color=keyword)) + geom_line()+ggtitle("Interest over time")+
        ylab("Relative Trend")+
        theme(plot.title = element_text(size = 18,colour="black"))+
        xlab('')+theme(axis.title.y = element_text(colour="#00007A",size=14,angle=90,hjust=.5,vjust=1),
                       axis.text.y = element_text(colour="darkred",size=14,angle=0,hjust=1,vjust=0),
                       axis.text.x = element_text(colour="darkred",size=14,angle=0,hjust=1,vjust=0))+
        theme(legend.title = element_text(colour="black", size=15, 
                                          face="bold"))+
        theme(legend.text = element_text(colour="blue", size=14, 
                                         face="bold"))
      
    }
    
  })
  
  
  corr<-reactive({
    
    if(input$corr==T & length(out()>1)){
      
      z=data()
      trend=z$trend
      trend=trend[,3:ncol(trend)]
      cor(trend)
      
    }
  }) 
  
  
  output$myplot3 <- renderPlot({
    if(length(corr()>0)){
      data=corr()
      
      qplot(x=Var1, y=Var2, data=melt(cor(data)), fill=value, geom="tile")+
        ggtitle('Correlation Matrix')+theme(axis.title.y =element_blank(),axis.title.x =element_blank(),
                                            axis.text.y = element_text(colour="darkred",size=14,angle=0,hjust=1,vjust=0),
                                            axis.text.x = element_text(colour="darkred",size=14,angle=0,hjust=1,vjust=0))+
        theme(legend.title=element_blank())+
        theme(legend.text = element_text(colour="black", size=14))+scale_fill_gradient2(limits=c(-1, 1),low="skyblue", high="blue")+
        theme(plot.title = element_text(size = 20,colour="black"))
    }
  })
  
  
  output$myplot2 <- renderPlot({
    if(length(out()>0)){
      data=data()
      
      
      z=data$searches
      rr=data$regions
      
      for (i in 1:length(z)){
        n=z[i]
        n=as.data.frame(n)
        names(n)=c("searches","hits")
        n$searches <- factor(n$searches, levels = n$searches[order(n$hits,decreasing =T)])
        
        colors=c("orange","skyblue","#999966")
        
        col=sample(c(1,2,3),1,replace=T)
        
        x11()
        
        print(ggplot(n, aes(searches,hits))+  
                geom_bar(stat='identity',fill=colors[col],color='black')+
                ggtitle(data$headers[2+2*length(z)+i])+ylab('Hits')+
                theme(plot.title = element_text(size = 18,colour="blue"))+
                theme(axis.title.x=element_blank(),axis.title.y = element_text(colour="blue",size=14),axis.text.x = element_text(colour="grey20",size=14,angle=60,hjust=.5,vjust=.5,face="plain"))
              
              
        )
        
        
        if(geo()=='')
        {
          x11()
          
          
          regions = as.data.frame(rr)[c(1,i+1)]
          
          names(regions)=c('region','hits')
          
          regions$region[regions$region=="United States"] = "USA"
          
          world_map = map_data("world")
          
          world_map =merge(world_map, regions, by="region",all.x = TRUE)
          
          world_map = world_map[order(world_map$group, world_map$order),]
          
          g=ggplot(world_map, aes(x=long, y=lat, group=group))+
            geom_polygon(aes(fill=hits), color="gray70") 
          
          print(g+theme(axis.text.y   = element_blank(),
                        axis.text.x   = element_blank(),
                        axis.title.y  = element_blank(),
                        axis.title.x  = element_blank(),
                        panel.background = element_blank(),
                        panel.grid.major = element_blank(), 
                        panel.grid.minor = element_blank())+
                  scale_fill_gradient(low = "skyblue", high = "blue", guide = "colorbar",na.value="white")+ggtitle(data$headers[2+2*length(z)+i])+ylab('Hits')+
                  theme(legend.key.size = unit(1, "cm"),
                        legend.title = element_text(size = 12, colour = "blue"),
                        legend.title.align=0.3,legend.text = element_text(size = 10))+
                  theme(panel.border = element_rect(colour = "gray70", fill=NA, size=0.5))
          )
        }
      }
      
    }
    
  }) 
  
}) 

<img style="margin:0px auto;display:block" src="C:/Users/agurm/Downloads/Personal Portfolio Website/Personal Portfolio Website/images/googletrend.PNG" ,="" height="370" width="900">


# Summary
In this tutorial, you discovered how to  used Google Trend API. 

Specifically, you learned:

- How to communicate with Google trend API.
- How to fetch a code to dvelop a model.